### Notes on Ch 6: Data Loading, Storage, and File Formats

Pandas provides various functions to read tabular data into a DataFrame. The most commonly used function is `pandas.read_csv`, which reads delimited data from a file, URL, or file-like object. Other functions are available for different formats, such as Excel, JSON, HDF5, and more.

#### Reading and Writing Data in Text Format

The `read_csv` function is extensively used to convert text data into a DataFrame. It has various optional arguments to handle different scenarios:

* <b>Indexing</b>

You can treat one or more columns as the returned DataFrame and specify whether to get column names from the file or from arguments you provide.

* <b>Type Inference and Data Conversion</b>

This involves user-defined value conversions and specifying a custom list of missing value markers.

* <b>Date and Time Parsing</b>

The `parse_dates` argument helps combine date and time information spread over multiple columns into a single column.

* <b>Date and Time Parsing</b>

Support for iterating over chunks of very large files is provided.

* <b>Unclean Data Issues</b>

Handling unclean data issues includes skipping rows or a footer, dealing with comments, and addressing numeric data with thousands separated by commas.



In [2]:
import pandas as pd

# Reading a CSV file with a header row
df = pd.read_csv("../data/ex1.csv")
df

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [5]:
# File without a Header

df_no_header = pd.read_csv("../data/ex2.csv", header=None)
print(df_no_header)
print("")
# Adding a header
df_custom_header = pd.read_csv("../data/ex2.csv", names=['a', 'b', 'c', 'd', 'message'])
print(df_custom_header)

   0   1   2   3      4
0  1   2   3   4  hello
1  5   6   7   8  world
2  9  10  11  12    foo

   a   b   c   d message
0  1   2   3   4   hello
1  5   6   7   8   world
2  9  10  11  12     foo


In [6]:
# Using a Column as an Index
df_indexed = pd.read_csv("../data/ex2.csv", names=['a', 'b', 'c', 'd', 'message'], index_col='message')
df_indexed

,a,b,c,d
message,,,,
hello,1,2,3,4
world,5,6,7,8
foo,9,10,11,12


In [8]:
# Hierarchical Indexing
parsed = pd.read_csv("../data/csv_mindex.csv", index_col=['key1', 'key2'])
parsed

value1  value2
key1 key2                
one  a          1       2
     b          3       4
     c          5       6
     d          7       8
two  a          9      10
     b         11      12
     c         13      14
     d         15      16

In [10]:
# Handling Whitespace-delimited File
result = pd.read_csv("../data/ex3.txt", sep='\s+')
result

,A,B,C
aaa,-0.264438,-1.026059,-0.619500
bbb,0.927272,0.302904,-0.032399
ccc,-0.264273,-0.386314,-0.217601
ddd,-0.871858,-0.348382,1.100491


In [11]:
# Skipping Rows
skipped_rows = pd.read_csv("../data/ex4.csv", skiprows=[0, 2, 3])
skipped_rows

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [13]:
# Handling Missing Values
result_missing = pd.read_csv("../data/ex5.csv")
print(result_missing)
print("")
result_custom_missing = pd.read_csv("../data/ex5.csv", na_values=['NULL'])
print(result_custom_missing)

  something  a   b     c   d message
0       one  1   2   3.0   4     NaN
1       two  5   6   NaN   8   world
2     three  9  10  11.0  12     foo

  something  a   b     c   d message
0       one  1   2   3.0   4     NaN
1       two  5   6   NaN   8   world
2     three  9  10  11.0  12     foo


#### Reading Text Files in Pieces

When dealing with large files, it may be practical to read only a small portion or iterate through chunks of the file. The `pandas.read_csv` function allows you to achieve this.

In this example, we read the entire file, then read only the first 5 rows, and finally, we read the file in chunks of 1000 rows, aggregating the value counts for the "key" column.

In [3]:
# Setting display to show only 10 rows for brevity
pd.options.display.max_rows = 10

# Reading the entire file
result = pd.read_csv("../data/ex6.csv")
print(result)
print("")

# Reading only a few rows
subset_result = pd.read_csv("../data/ex6.csv", nrows=5)
print(subset_result)
print("")

# Reading the file in chunks (chunksize=1000)
chunker = pd.read_csv("../data/ex6.csv", chunksize=1000)
print(type(chunker)) # Returns a TextFileReader object
print("")

# Iterating over the chunks
tot = pd.Series([], dtype='int64')

for piece in chunker:
    tot = tot.add(piece["key"].value_counts(), fill_value=0)

tot = tot.sort_values(ascending=False)
print(tot[:10])

           one       two     three      four key
0     0.467976 -0.038649 -0.295344 -1.824726   L
1    -0.358893  1.404453  0.704965 -0.200638   B
2    -0.501840  0.659254 -0.421691 -0.057688   G
3     0.204886  1.074134  1.388361 -0.982404   R
4     0.354628 -0.133116  0.283763 -0.837063   Q
...        ...       ...       ...       ...  ..
9995  2.311896 -0.417070 -1.409599 -0.515821   L
9996 -0.479893 -0.650419  0.745152 -0.646038   E
9997  0.523331  0.787112  0.486066  1.093156   K
9998 -0.362559  0.598894 -1.843201  0.887292   G
9999 -0.096376 -1.012999 -0.657431 -0.573315   0

[10000 rows x 5 columns]

        one       two     three      four key
0  0.467976 -0.038649 -0.295344 -1.824726   L
1 -0.358893  1.404453  0.704965 -0.200638   B
2 -0.501840  0.659254 -0.421691 -0.057688   G
3  0.204886  1.074134  1.388361 -0.982404   R
4  0.354628 -0.133116  0.283763 -0.837063   Q

<class 'pandas.io.parsers.readers.TextFileReader'>

key
E    368.0
X    364.0
L    346.0
O    343.0
Q    340

#### Writing Data to Text Format

You can export data from a DataFrame to a delimited format using the to_csv method. Here's an example:

In [4]:
# Reading a CSV file
df = pd.read_csv("../data/ex5.csv")
df

# Writing a CSV file
df.to_csv("../data/out.csv")

This code reads a CSV file into a DataFrame and then writes the DataFrame back to a new CSV file named "out.csv."

You can customize the delimiter, handle missing values, and control whether to include index and header information in the output. Here are a few examples:

In [8]:
import sys

# Writing data with a different delimiter (pipe '|')
df.to_csv(sys.stdout, sep='|')
print("")

# Writing data with a specific representation for missing values
df.to_csv(sys.stdout, na_rep="NULL")
print("")

# Writing data without including index and header
df.to_csv(sys.stdout, index=False, header=False)
print("")

# Writing only a subset of columns
df.to_csv(sys.stdout, index=False, columns=["a", "b", "c"])

|something|a|b|c|d|message
0|one|1|2|3.0|4|
1|two|5|6||8|world
2|three|9|10|11.0|12|foo

,something,a,b,c,d,message
0,one,1,2,3.0,4,NULL
1,two,5,6,NULL,8,world
2,three,9,10,11.0,12,foo

one,1,2,3.0,4,
two,5,6,,8,world
three,9,10,11.0,12,foo

a,b,c
1,2,3.0
5,6,
9,10,11.0


#### Working with Other Delimited Formats

If you encounter a file with a format not easily handled by `pandas.read_csv`, you can resort to Python's built-in csv module. Here's a step-by-step example:

In [10]:
import csv

# Reading a CSV file using csv.reader
##  First, we read the file into a list of lines:
with open("../data/ex7.csv") as f:
    lines = list(csv.reader(f))

# Separating header and values
header, values = lines[0], lines[1:]

# Creating a dictionary of data columns
data_dict = {h: v for h, v in zip(header, zip(*values))}
print(data_dict)

{'a': ('1', '1'), 'b': ('2', '2'), 'c': ('3', '3')}


You can also define your CSV dialect using a subclass of csv.Dialect or provide individual options as keywords. The possible options include delimiter, lineterminator, quotechar, quoting, skipinitialspace, and doublequote. For more complicated cases, manual parsing using string methods or regular expressions might be necessary.

In [12]:
# Defining a custom CSV dialect
class my_dialect(csv.Dialect):
    lineterminator = "\n"
    delimiter = ";"
    quotechar = '"'
    quoting = csv.QUOTE_MINIMAL

# Using the custom dialect with csv.reader
#reader = csv.reader(f, dialect=my_dialect)

# Alternatively, using individual CSV dialect parameters
#reader = csv.reader(f, delimiter="|")